In [31]:
def extract_json_content(text):
    start = text.find('{')  # Find the index of the first '['
    end = text.rfind('}')   # Find the index of the last ']'

    # Extract the JSON content between the first '[' and the last ']'
    json_content = text[start:end + 1]

    # Extract the JSON content
    if start == -1 or end == -1:
        raise ValueError("No JSON object found in the text.")
    return text[start:end+1]

In [2]:
from llm_response import MentalHealthBot
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import json
import re
from tenacity import retry, stop_after_attempt, wait_exponential

class MentalHealthRAGEvaluator:
    def __init__(self, k=3, test_size=0.2, random_state=42):
        self.bot = MentalHealthBot()
        self.vectorizer = TfidfVectorizer()
        self.k = k
        self.test_size = test_size
        self.random_state = random_state
        
    @retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
    def get_bot_response(self, prompt):
        try:
            response = self.bot.chat_stream(prompt)
            json_content = re.search(r'\{.*\}', response, re.DOTALL)
            return json.loads(json_content.group()) if json_content else None
        except Exception as e:
            print(f"Error getting response: {str(e)}")
            raise

    def evaluate(self, data_path):
        df = pd.read_csv(data_path).dropna()
        train_df, test_df = train_test_split(
            df, test_size=self.test_size, random_state=self.random_state
        )
        
        # TF-IDF Vectorization
        train_texts = train_df.iloc[:, 1].tolist()
        train_labels = train_df.iloc[:, 2].tolist()
        X_train = self.vectorizer.fit_transform(train_texts)
        
        # Nearest Neighbors
        self.nn = NearestNeighbors(n_neighbors=self.k).fit(X_train)
        
        # Evaluation
        y_true, y_pred = [], []
        for _, row in test_df.head(500).iterrows():
            query = row.iloc[1]
            true_label = row.iloc[2].lower()
            y_true.append(true_label)
            
            try:
                # Find similar examples using TF-IDF
                query_vec = self.vectorizer.transform([query])
                _, indices = self.nn.kneighbors(query_vec)
                
                # Create prompt
                prompt = "Similar examples:\n"
                for i in indices[0]:
                    prompt += f"- {train_texts[i][:150]}... [Label: {train_labels[i]}]\n"
                prompt += f"\nClassify this:\n{query[:300]}\nCategory:"
                
                # Get prediction
                response = self.get_bot_response(prompt)
                pred = (response.get('Category') or response.get('category')).lower()
                y_pred.append(pred)
                
            except Exception as e:
                print(f"Error processing: {str(e)}")
                y_pred.append("error")
        
        # Calculate metrics
        valid_indices = [i for i, pred in enumerate(y_pred) if pred != "error"]
        if valid_indices:
            y_true_valid = [y_true[i] for i in valid_indices]
            y_pred_valid = [y_pred[i] for i in valid_indices]
            
            print("\nEvaluation Metrics:")
            print(f"Accuracy: {accuracy_score(y_true_valid, y_pred_valid):.4f}")
            print(f"Macro F1: {f1_score(y_true_valid, y_pred_valid, average='macro'):.4f}")
            print(classification_report(y_true_valid, y_pred_valid, zero_division=0))

if __name__ == "__main__":
    evaluator = MentalHealthRAGEvaluator(k=5)
    evaluator.evaluate("Combined-Data.csv")

Starting Bot -----------------------------------###
Initializing RAG system
Initializing LLM
Creating RAG chain


KeyboardInterrupt: 